In [45]:
import pandas as pd

from bs4 import BeautifulSoup
import requests

from keybert import KeyBERT
from tqdm import tqdm

In [ ]:
#UCD 
ucd_url = "https://catalog.ucdavis.edu/courses-subject-code/ecs/"
req = requests.get(ucd_url)
soup = BeautifulSoup(req.text)

divs = soup.find_all("div", class_="courseblock")

class_id = []
class_title = []
desc = []


for div in divs:
    code_element = div.find("span", class_="text courseblockdetail detail-code margin--span text--semibold text--big")
    title_element = div.find("span", class_="text courseblockdetail detail-title margin--span text--semibold text--big")
    desc_element = div.find("p", class_= "courseblockextra noindent")

    class_id.append(code_element.text.strip())
    class_title.append(title_element.text.strip().replace('—', ""))
    desc.append(desc_element.text.split("Course Description:")[1])

df = pd.DataFrame({"Course ID": class_id, "Course Title": class_title, "Course Description":desc})

df["Upper Div"] = df["Course ID"].str.extract(r'(\d+)')[0].astype(int).apply(lambda x:x>=100)
df = df[df["Course ID"].str.extract(r'(\d+)')[0].astype(int) < 200]


df




,Course ID,Course Title,Course Description,Upper Div
0,ECS 011,Artificial Intelligence for All,Comprehensive introduction to artificial inte...,False
1,ECS 012,Introduction to Media Computation,Introduction to key computational ideas neces...,False
2,ECS 017,"Data, Logic, & Computing","Display, processing, and representation of in...",False
3,ECS 020,Discrete Mathematics For Computer Science,Discrete mathematics of particular utility to...,False
4,ECS 032A,Introduction to Programming,Introduction to programming and problem solvi...,False
...,...,...,...,...
94,ECS 198,Directed Group Study,Directed group study.,True
95,ECS 198F,Student Facilitated Course,Student facilitated course intended primarily...,True
96,ECS 199,Special Study for Advanced Undergraduates,Special study for advanced undergraduates.,True
97,ECS 199FA,Student Facilitated Course Development,"Under the supervision of a faculty member, an...",True


In [49]:
def keyword_wrapper(doc):
    kw_model = KeyBERT() #instantiate model
    
    stop_words = ["cs", "prerequisite", "grade", "requirement", 
                  "courses", "instructor", "faculty", "computer", "student", "concurrently", "majors"] #dont consider these words

    return [i[0] for i in kw_model.extract_keywords(doc, stop_words=stop_words, top_n=10)] #top 10 keywords

df['keywords'] = df['Course Description'].progress_apply(keyword_wrapper) #apply functon
df

100%|██████████| 99/99 [02:26<00:00,  1.48s/it]


,Course ID,Course Title,Course Description,Upper Div,keywords
0,ECS 011,Artificial Intelligence for All,Comprehensive introduction to artificial inte...,False,"[ai, intelligence, ethical, awareness, introdu..."
1,ECS 012,Introduction to Media Computation,Introduction to key computational ideas neces...,False,"[programming, computational, digital, media, f..."
2,ECS 017,"Data, Logic, & Computing","Display, processing, and representation of in...",False,"[computers, algorithms, mathematics, processin..."
3,ECS 020,Discrete Mathematics For Computer Science,Discrete mathematics of particular utility to...,False,"[mathematics, discrete, combinatorics, recursi..."
4,ECS 032A,Introduction to Programming,Introduction to programming and problem solvi...,False,"[python, programming, students, introduction, ..."
...,...,...,...,...,...
94,ECS 198,Directed Group Study,Directed group study.,True,"[group, study, directed]"
95,ECS 198F,Student Facilitated Course,Student facilitated course intended primarily...,True,"[students, facilitated, division, primarily, c..."
96,ECS 199,Special Study for Advanced Undergraduates,Special study for advanced undergraduates.,True,"[undergraduates, study, advanced, special, for]"
97,ECS 199FA,Student Facilitated Course Development,"Under the supervision of a faculty member, an...",True,"[098f, undergraduate, 198f, plans, supervision..."


In [53]:
df.to_csv("/tmp/ucd.csv", index = False)